# 2.5 Project: Digit Classifier with GUI (Tkinter)

This project demonstrates how to build a professional digit classifier application with a graphical user interface (GUI) using Tkinter and a trained neural network. The goal is to bridge the gap between educational prototypes and real-world deployment.

## Learning Objectives
- Integrate a trained neural network model into a desktop GUI application.
- Use Tkinter for drawing and user interaction.
- Preprocess user input for model prediction.
- Discuss best practices for packaging and deployment.
---

## Overview of the Architecture
A professional ML application typically consists of:
- **Frontend/UI**: User interaction (drawing digits, displaying predictions)
- **Backend/Model**: Loads and runs the trained neural network
- **Preprocessing**: Converts user input to the format expected by the model
- **Packaging**: Bundling dependencies for distribution (e.g., with PyInstaller)


## Step 1: Train and Save a Digit Classifier Model
We will use TensorFlow/Keras to train a simple MLP on MNIST and save the model for later use in the GUI.

In [11]:
# Train and save a digit classifier model
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
model = Sequential([
    Flatten(input_shape=(28,28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test), verbose=2)
model.save('my_model.keras')

/media/kmom/Data/github/kmom88/zero-to-hero-AI/venv/lib/python3.10/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1875/1875 - 3s - 2ms/step - accuracy: 0.9251 - loss: 0.2601 - val_accuracy: 0.9598 - val_loss: 0.1371
Epoch 2/5
1875/1875 - 3s - 2ms/step - accuracy: 0.9251 - loss: 0.2601 - val_accuracy: 0.9598 - val_loss: 0.1371
Epoch 2/5
1875/1875 - 3s - 1ms/step - accuracy: 0.9669 - loss: 0.1121 - val_accuracy: 0.9688 - val_loss: 0.1026
Epoch 3/5
1875/1875 - 3s - 1ms/step - accuracy: 0.9669 - loss: 0.1121 - val_accuracy: 0.9688 - val_loss: 0.1026
Epoch 3/5
1875/1875 - 2s - 1ms/step - accuracy: 0.9762 - loss: 0.0781 - val_accuracy: 0.9754 - val_loss: 0.0840
Epoch 4/5
1875/1875 - 2s - 1ms/step - accuracy: 0.9762 - loss: 0.0781 - val_accuracy: 0.9754 - val_loss: 0.0840
Epoch 4/5
1875/1875 - 3s - 1ms/step - accuracy: 0.9826 - loss: 0.0580 - val_accuracy: 0.9748 - val_loss: 0.0778
Epoch 5/5
1875/1875 - 3s - 1ms/step - accuracy: 0.9826 - loss: 0.0580 - val_accuracy: 0.9748 - val_loss: 0.0778
Epoch 5/5
1875/1875 - 2s - 1ms/step - accuracy: 0.9862 - loss: 0.0458 - val_accuracy: 0.9756 - val_loss:

---
## Step 2: Build the Tkinter GUI
The GUI will allow users to draw a digit, preprocess the image, and display the model's prediction.
In ubuntu 22.04, you can install Tkinter with:
```bash
sudo apt install python3-tk
```

In [12]:
# Tkinter GUI for digit classification
import tkinter as tk
from PIL import Image, ImageDraw, ImageOps
import numpy as np
import tensorflow as tf

class DigitClassifierApp:
    def __init__(self, master):
        self.master = master
        self.master.title('Digit Classifier')

        # Canvas para dibujar
        self.canvas = tk.Canvas(master, width=280, height=280, bg='white')
        self.canvas.pack()

        # Botones
        self.button_predict = tk.Button(master, text='Predict', command=self.predict_digit)
        self.button_predict.pack()

        self.button_clear = tk.Button(master, text='Clear', command=self.clear_canvas)
        self.button_clear.pack()

        # Label de resultado
        self.label_result = tk.Label(master, text='Draw a digit and click Predict')
        self.label_result.pack()

        # Imagen PIL para dibujar
        self.image = Image.new('L', (280, 280), 'white')
        self.draw = ImageDraw.Draw(self.image)

        # Evento de dibujo
        self.canvas.bind('<B1-Motion>', self.paint)

        # Cargar modelo
        self.model = tf.keras.models.load_model('my_model.keras')

    def paint(self, event):
        x1, y1 = (event.x - 8), (event.y - 8)
        x2, y2 = (event.x + 8), (event.y + 8)
        self.canvas.create_oval(x1, y1, x2, y2, fill='black', width=0)
        self.draw.ellipse([x1, y1, x2, y2], fill='black')

    def predict_digit(self):
        img = self.image.resize((28, 28))
        img = ImageOps.invert(img)
        img = np.array(img) / 255.0
        img = img.reshape(1, 28, 28)
        pred = self.model.predict(img)
        digit = np.argmax(pred)
        self.label_result.config(text=f'Prediction: {digit}')

    def clear_canvas(self):
        # Limpia el canvas
        self.canvas.delete('all')
        # Reinicia la imagen PIL
        self.image = Image.new('L', (280, 280), 'white')
        self.draw = ImageDraw.Draw(self.image)
        # Restablece el mensaje
        self.label_result.config(text='Draw a digit and click Predict')

if __name__ == '__main__':
    root = tk.Tk()
    app = DigitClassifierApp(root)
    root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


---
## Professional Packaging and Deployment
For real-world use, consider:
- Packaging with PyInstaller or cx_Freeze to create standalone executables.
- Including model files and dependencies in the package.
- Adding error handling and logging.
- Designing a clean, user-friendly interface.
- Optionally, using ONNX for cross-framework model deployment.

---
## Summary and Next Steps
- You have built a digit classifier with a GUI using Tkinter and a trained neural network.
- This approach can be extended to other image or text classification tasks.
- For professional projects, focus on maintainability, packaging, and user experience.
- Next: Try deploying your app, or extend it with more advanced models and features!
---